In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


In [12]:
# 1. download html with a get request
headers = {'Accept-Language': 'en-US,en;q=0.8'}


In [13]:
# 2. find url and store it in a variable
url_berlin = "https://en.wikipedia.org/wiki/Berlin"


In [14]:
berlin = requests.get(url_berlin)
berlin.status_code

200

In [15]:
soup_berlin = BeautifulSoup(berlin.content, "html.parser")


In [16]:
BR_city=soup_berlin.select(".mw-page-title-main")[0].get_text()

In [17]:
BR_country=soup_berlin.select('a[href="/wiki/Germany"]')[0].get_text()

In [18]:
BR_latitude=soup_berlin.select(".latitude")[0].get_text()
BR_latitude

'52°31′12″N'

In [19]:
BR_longitude=soup_berlin.select(".longitude")[0].get_text()

In [20]:
BR_population=soup_berlin.select('td.infobox-data')[10].get_text()


In [14]:
# Frankfurt

In [21]:
url_frankfurt = "https://en.wikipedia.org/wiki/Frankfurt"
frankfurt = requests.get(url_frankfurt)
frankfurt.status_code


200

In [22]:
soup_frankfurt = BeautifulSoup(frankfurt.content, "html.parser")


In [23]:
FR_city=soup_frankfurt.select(".mw-page-title-main")[0].get_text()

In [24]:
FR_country=soup_frankfurt.select('a[href="/wiki/Germany"]')[0].get_text()
FR_country

'Germany'

In [25]:
FR_latitude=soup_frankfurt.select(".latitude")[0].get_text()

In [39]:
FR_longitude=soup_frankfurt.select(".longitude")[0].get_text()

In [40]:
FR_population=soup_frankfurt.select("td.infobox-data")[10].get_text()


In [22]:
# Hamburg

In [28]:
#url_hamburg = "https://en.wikipedia.org/wiki/Hamburg"
#hamburg = requests.get(url_hamburg)
#hamburg.status_code

In [29]:
#soup_hamburg = BeautifulSoup(hamburg.content, "html.parser")


In [25]:
#HB_latitude=soup_hamburg.select(".latitude")[0].get_text()

In [30]:
#HB_longitude=soup_hamburg.select(".longitude")[0].get_text()

In [31]:
#HB_population=soup_hamburg.select("td.infobox-data")[8].get_text()
#<td class="infobox-data">1,945,532</td>

In [32]:
#HB_country= soup_hamburg.select("td.infobox-data")[0].get_text()
#HB_country
#<td class="infobox-data">Germany</td>

In [33]:
#HB_city=soup_hamburg.select(".mw-page-title-main")[0].get_text()
#HB_city

In [30]:
#DataFrame

In [41]:
# Create a DataFrame
data = {
    "City": [FR_city, BR_city],
    "Population": [FR_population, BR_population],
    "Latitude": [FR_latitude, BR_latitude],
    "Longitude": [FR_longitude, BR_longitude],
    "Country": [FR_country, BR_country]
}

df = pd.DataFrame(data)


In [42]:
df

,City,Population,Latitude,Longitude,Country
0,Frankfurt,"773,068",50°06′38″N,08°40′56″E,Germany
1,Berlin,"3,850,809",52°31′12″N,13°24′18″E,Germany


In [44]:
# Fixing latitude
df['Latitude'] = df['Latitude'].str.split('″').str[0].str.replace('°', '.').str.replace('′', '')

# Fixing longitude
df['Longitude'] = df['Longitude'].str.split('″').str[0].str.replace('°', '.').str.replace('′', '')

# Fixing population
df["Population"] = df["Population"].str.replace(',', '')

# Remove "N" and "E" suffixes from Latitude and Longitude columns
df['Latitude'] = df['Latitude'].str.rstrip('N')
df['Longitude'] = df['Longitude'].str.rstrip('E')

# Print the DataFrame
df

,City,Population,Latitude,Longitude,Country
0,Frankfurt,773068,50.0638,08.4056,Germany
1,Berlin,3850809,52.3112,13.2418,Germany


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        3 non-null      object 
 1   Population  3 non-null      int64  
 2   Latitude    3 non-null      float64
 3   Longitude   3 non-null      float64
 4   Country     3 non-null      object 
dtypes: float64(2), int64(1), object(2)
memory usage: 248.0+ bytes


In [37]:
# Convert 'Population', 'Latitude', and 'Longitude' columns to numeric types
df['Population'] = pd.to_numeric(df['Population'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['City'] = df['City'].astype(str)

# Check the data types after conversion
print(df.dtypes)


City           object
Population      int64
Latitude      float64
Longitude     float64
Country        object
dtype: object


In [12]:
#!pip install sqlalchemy
import sqlalchemy

schema="gans"
host="localhost"
user="root"
password="NelaDurres88"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'




In [ ]:
df.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)


In [13]:
# Implementing the code with Fucntions

In [38]:
def scrape_city_data(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        city = soup.title.get_text().split(' - ')[0]
        country = soup.select('td.infobox-data a')[0].get_text()
        latitude = soup.select('span.latitude')[0].get_text()
        longitude = soup.select('span.longitude')[0].get_text()

        # Find the population data using provided code
        population_element = soup.select_one('th.infobox-header:-soup-contains("Population")')
        if population_element:
            population = population_element.parent.find_next_sibling().find(string=re.compile(r'\d+'))
            if population:
                population = int(population)
        else:
            population = None

        data = {
            'City': [city],
            'Country': [country],
            'Latitude': [latitude],
            'Longitude': [longitude],
            'Population': [population],
        }

        city_df = pd.DataFrame(data)
        return city_df

    else:
        print("Error:", response.status_code)
        return None

# List of German cities
german_cities = ['Berlin', 'Frankfurt']

# Create an empty DataFrame with specified columns
german_cities_df = pd.DataFrame(columns=['City', 'Country', 'Latitude', 'Longitude', 'Population'])

# Iterate and scrape data for German cities
for city_name in german_cities:
    wiki_link = f"https://en.wikipedia.org/wiki/{city_name}"
    city_data = scrape_city_data(wiki_link)

    # Append the data to the table
    if city_data is not None:
        german_cities_df = pd.concat([german_cities_df, city_data], ignore_index=True)

# Display the DataFrame
print(german_cities_df)
 

NotImplementedError: ':-soup-contains' pseudo-class is not implemented at this time